In [164]:
import pandas as pd
import numpy as np
import re
import nltk
# import warnings 
# warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

from html import unescape
import json

In [54]:
train = pd.read_csv('../data/train_tweets.csv')
test = pd.read_csv('../data/test_tweets.csv')

In [55]:
data = pd.concat([train, test]).reset_index().drop('index', axis=1)

In [57]:
data.tail()

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) & u...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ..."


1. Заменим html-сущности (к примеру: `&lt;` `&gt;` `&amp;`). `&lt;` заменим на `<` и `&amp;` заменим на `&`). Сделаем это с помощью HTMLParser.unescape()

In [62]:
data.tweet = data.tweet.apply(lambda x: unescape(x))

In [63]:
data.tweet.str.contains('&lt;').any()

False

2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


В [документации](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html) `numpy` говорится, что `vectorize` просто для удобства, а не производительности, поэтому можно использовать `apply` в `pandas` с тем же успехом.

```The vectorize function is provided primarily for convenience, not for performance. The implementation is essentially a for loop.```

In [71]:
def remove_handles(s):
    return re.sub(r'@[\w]+', '', s)

In [78]:
data.tweet = data.tweet.apply(lambda x: remove_handles(x))

3. Изменим регистр твитов на нижний с помощью .lower()

In [80]:
data.tweet = data.tweet.str.lower()

In [68]:
def load_dict(path):
    with open(path) as file:
        data = json.load(file)
    return data

4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [93]:
apostrophe_dict = load_dict('../data/apostrophe_dict.json')

In [94]:
def replace_dict(token, dictionary):
    return dictionary.get(token, token)

In [95]:
data['tokens'] = data.tweet.apply(lambda x: x.split())

In [103]:
data.tokens = data.tokens.apply(lambda x: [replace_dict(token, apostrophe_dict) for token in x])

5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [106]:
short_word_dict = load_dict('../data/short_word_dict.json')

In [107]:
data.tokens = data.tokens.apply(lambda x: [replace_dict(token, short_word_dict) for token in x])

6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [110]:
emoticon_dict = load_dict('../data/emotion_dict.json')

In [111]:
data.tokens = data.tokens.apply(lambda x: [replace_dict(token, emoticon_dict) for token in x])

7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [119]:
data['tweets_processed'] = data.tokens.apply(lambda x: ' '.join(x))

In [123]:
data.tweets_processed = data.tweets_processed.apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [124]:
data.tweets_processed = data.tweets_processed.apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))

9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [125]:
data.tweets_processed = data.tweets_processed.apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [128]:
data.tweets_processed = data.tweets_processed.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 1]))

11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [161]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [132]:
data['tweet_token'] = data.tweets_processed.apply(lambda x: nltk.word_tokenize(x))

12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [143]:
stop_words = nltk.corpus.stopwords.words('english')

In [144]:
data['tweet_token_filtered' ] = data.tweet_token.apply(lambda x: [w for w in x if w not in stop_words])

13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [153]:
stemmer = nltk.stem.PorterStemmer()

In [155]:
data['tweet_stemmed'] = data.tweet_token_filtered.apply(lambda x: [stemmer.stem(w) for w in x])

14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [156]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

In [162]:
data['tweet_lemmatized'] = data.tweet_token_filtered.apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

15. Сохраним результат предобработки в pickle-файл.

In [165]:
data.to_pickle('../data/tweet_data.pkl')

_Примечание:_ Задание выполнил по пунктам. При этом логика в последовательности действий нарушена. В частности токенизация с помощью `word_tokenize()` после выполненых ранее действий никакой дополнительной информации по сравнению со `str.split()` не даёт.